In [2]:
import pandas as pd
import numpy as np
import math
import json
import csv

In [4]:
df = pd.read_csv('processed_routes_big.csv')
stllf = open('translated_stations_normalized.json')
st_with_latlong = json.load(stllf)

In [ ]:
def get_lat_long(station_name_en: str, city_name_en: str, region_id: int):
    pref = str(region_id)[1:]
    for st in st_with_latlong:
        if station_name_en == st['name_romaji'] and pref == st['prefecture'] and city_name_en == st['city'].lower():
            return st['stations'][0]['lat'], st['stations'][0]['lon']
    print(f'For {station_name_en} in {city_name_en}, region {pref}, not found, returning null values')
    return None, None

In [9]:
def fetch_from_station_lat_lon(row):
    station_name = row['from_station']
    city_name = row['from_city']
    region_id = row['from_region_id']
    return get_lat_long(station_name, city_name, region_id)

# Apply the function and store results in new columns
df[['from_lat', 'from_lon']] = df.apply(
    lambda row: pd.Series(fetch_from_station_lat_lon(row)), axis=1
)

In [10]:
def fetch_to_station_lat_lon(row):
    station_name = row['to_station']
    city_name = row['to_city']
    region_id = row['to_region_id']
    return get_lat_long(station_name, city_name, region_id)

# Apply the function and store results in new columns
df[['to_lat', 'to_lon']] = df.apply(
    lambda row: pd.Series(fetch_to_station_lat_lon(row)), axis=1
)

In [39]:
print(df.head().to_markdown())

|    | from_city   |   from_region_id | from_region_name   | to_city   |   to_region_id | to_region_name   | type   | from_station   | to_station   |   duration |   distance |   cost | route_id    | route_name   | enabled   |   from_lat |   from_lon |   to_lat |   to_lon |
|---:|:------------|-----------------:|:-------------------|:----------|---------------:|:-----------------|:-------|:---------------|:-------------|-----------:|-----------:|-------:|:------------|:-------------|:----------|-----------:|-----------:|---------:|---------:|
|  0 | tokyo       |              213 | tokyo              | tokyo     |            213 | tokyo            | train  | Tokyo          | Ueno         |        nan |     3.7356 |    nan | JPRail 1009 | 北陸新幹線   | True      |    35.6814 |    139.766 |  35.7138 |  139.777 |
|  1 | tokyo       |              213 | tokyo              | tokyo     |            213 | tokyo            | train  | Ueno           | Tokyo        |        nan |     3.7356 |    nan 

In [21]:
import math

def haversine(lat1, lon1, lat2, lon2):
    # Convert latitude and longitude from degrees to radians
    lat1, lon1, lat2, lon2 = map(math.radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = math.sin(dlat / 2)**2 + math.cos(lat1) * math.cos(lat2) * math.sin(dlon / 2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    R = 6371  # Radius of Earth in kilometers
    distance = R * c  # Distance in kilometers

    return distance


In [25]:
def sanity():
    # Exclude specific columns from null check
    subset_cols = [col for col in df.columns if col not in ['duration', 'distance', 'cost']]

    # Count null rows in the subset
    null_count = df[subset_cols].isnull().sum()

    # Print the total number of null rows
    print(f"Total null rows (excluding duration, distance, and cost): {null_count.sum()}")

def sanity_1():
    # Exclude specific columns from null check
    subset_cols = [col for col in df.columns if col in ['distance']]

    # Count null rows in the subset
    null_count = df[subset_cols].isnull().sum()

    # Print the total number of null rows
    print(f"Total null rows in distance: {null_count.sum()}")

def sanity_2():
    subset_cols = [col for col in df.columns if col in ['duration']]

    # Count null rows in the subset
    null_count = df[subset_cols].isnull().sum()

    # Print the total number of null rows
    print(f"Total null rows in duration: {null_count.sum()}")

In [23]:
sanity()

Total null rows (excluding duration, distance, and cost): 0


In [24]:
# Calculate distance using haversine
df["distance"] = df.apply(
    lambda row: haversine(row["from_lat"], row["from_lon"], row["to_lat"], row["to_lon"]), axis=1
)

In [26]:
sanity_1()

Total null rows in distance: 0


In [100]:
def calculate_effective_speed(distance, max_speed, short_threshold=10, medium_threshold=50):
    if distance <= short_threshold:
        # Slow intra-city travel
        return 65
    elif distance <= medium_threshold:
        # Medium-distance adjustment
        return 120
    else:
        # Long-distance, use max speed
        return max_speed

def estimate_duration(distance, route_id):
    route = int(route_id.split(' ')[1])
    factor = 1.25
    speed = 60 if route > 10000 else calculate_effective_speed(distance, 250, 10, 40)
    # print(speed)
    return math.floor(distance * factor / (speed / 60)) + 1

In [101]:
estimate_duration( 25.4797 , 'JPRailway 1002')

16

In [102]:
df["duration"] = df.apply(
    lambda row: estimate_duration(row['distance'], row['route_id']), axis=1
)

In [103]:
sanity_2()

Total null rows in duration: 0


In [104]:
print(df.head(100).to_markdown())

|    | from_city    |   from_region_id | from_region_name   | to_city      |   to_region_id | to_region_name   | type   | from_station         | to_station           |   duration |   distance |   cost | route_id     | route_name                       | enabled   |   from_lat |   from_lon |   to_lat |   to_lon |
|---:|:-------------|-----------------:|:-------------------|:-------------|---------------:|:-----------------|:-------|:---------------------|:---------------------|-----------:|-----------:|-------:|:-------------|:---------------------------------|:----------|-----------:|-----------:|---------:|---------:|
|  0 | tokyo        |              213 | tokyo              | tokyo        |            213 | tokyo            | train  | Tokyo                | Ueno                 |          5 |    3.7356  |    nan | JPRail 1009  | 北陸新幹線                       | True      |    35.6814 |    139.766 |  35.7138 |  139.777 |
|  1 | tokyo        |              213 | tokyo              | toky

In [106]:
df.to_csv('processed_routes_big.csv', index=False)